# Handwritten Digit Recognition with Machine Learning

Just attempting to try handwritten digit recognition with machine learning.

Project inspired by scikit-learn's "Recognizing hand-written digits": https://scikit-learn.org/stable/auto_examples/classification/plot_digits_classification.html#sphx-glr-auto-examples-classification-plot-digits-classification-py

## Import libraries

In [ ]:
import numpy as np
from numpy import asarray
from tensorflow.keras.datasets import mnist

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

import tkinter as tk
from PIL import Image
from PIL import ImageGrab

## Import the data sets and labels. 

In [ ]:
train_data, train_labels, test_data, test_labels = mnist.load_data()

#print('Train Data: ', train_data.shape)
#print('Train Labels: ', train_labels.shape)

# To apply a classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = train_data.shape[0]
train_data = train_data.reshape((n_samples, -1))

## Training the models (and getting the predictions).

In [ ]:
#----------Random Forest----------
rf_model = RandomForestClassifier(n_estimators = 100)
rf_model.fit(train_data, train_labels)
#rf_pred = rf_model.predict(test_data) 

In [ ]:
#----------Neural Network (MLP)----------
nn_model = MLPClassifier(hidden_layer_sizes=(100, ), max_iter = 1000).fit(train_data, train_labels)
#nn_pred= nn_model.predict(test_data)

## View the performance

In [ ]:
print("Random Forest Performance in %:")
print("Accuracy:", accuracy_score(test_labels, rf_pred) * 100)
print("Precision:", precision_score(test_labels, rf_pred, average = "weighted") * 100)
print("Recall:", recall_score(test_labels, rf_pred, average = "weighted") * 100)
print("F1 Score:", f1_score(test_labels, rf_pred, average = "weighted") * 100)
print()

In [ ]:
print("Neural Network Performance in %:")
print("Accuracy:", accuracy_score(test_labels, nn_pred) * 100)
print("Precision:", precision_score(test_labels, nn_pred, average = "weighted") * 100)
print("Recall:", recall_score(test_labels, nn_pred, average = "weighted") * 100)
print("F1 Score:", f1_score(test_labels, nn_pred, average = "weighted") * 100)

## Testing image imports 

In [ ]:
# Converts it to a single colour depth.
img = Image.open('two.png').convert('P', palette = Image.ADAPTIVE, colors = 50)
img.thumbnail((28, 28), Image.ANTIALIAS)
img.show()
npImg = asarray(img).reshape(1, -1)

pred = rf_model.predict(npImg)
print(pred)

In [ ]:
print(npImg.shape)
plt.imshow(img)
plt.show()

## GUI For Canvas 

<b>NOTE:</b> BEFORE RUNNING THE CODE, go on Display Settings -> Scale and layout -> then "Change the size of text, apps and other items" to 100%. Otherwise, fetching the co-ordinates of the canvas will be inaccurate.

In [ ]:
canvasWidth = 280
canvasHeight = 280
mousePos = { "x":0, "y":0 }
lineWidth = 15

# Sets the current position of the mouse.
def setPosition(event):
    mousePos["x"] = event.x 
    mousePos["y"] = event.y

# Draw on the canvas based on the current position of the mouse.
def draw(event):
    currX = mousePos["x"]
    currY = mousePos["y"]
    canvas.create_oval(currX, currY, currX, currY, width = lineWidth)
    setPosition(event)

# Removes the canvas content.
def clearCanvas():
    canvas.delete("all")

# Takes a screenshot of the canvas and stores it as an image.
def saveImage():
    canvas.update()
    
    # Added (+2) to prevent the border from being screenshotted.
    x0 = canvas.winfo_rootx() + 2
    y0 = canvas.winfo_rooty() + 2
    x1 = x0 + canvasWidth
    y1 = y0 + canvasHeight
    
    screenShot = ImageGrab.grab((x0, y0, x1, y1))
    
    # Converts it to a single colour depth.
    canvasImg = screenShot.convert('P', palette = Image.ADAPTIVE, colors = 10)
    canvasImg.thumbnail((28, 28), Image.ANTIALIAS) # (28,28) is the size of the train/test images.
    predictImg(canvasImg)
    
# Makes the predictions on the image.
def predictImage(img):
    npImg = asarray(img).reshape(1, -1)
    rf_pred = rf_model.predict(npImg)
    #nn_pred = nn_model.predict(npImg)
    
    # Display them on the GUI.
    lblRandForest.config(text = "Random Forest: " + str(rf_pred))
    #lblNeuralNet.config(text = "Neural Network: " + str(nn_pred))

# Create the window and add its elements.
root = tk.Tk()
root.geometry("500x500")
root.resizable(0, 0)

canvas = tk.Canvas(root, bg = "white", width = canvasWidth, height = canvasHeight, 
                   highlightthickness = 2, highlightbackground = "black")
canvas.bind("<Button-1>", setPosition)
canvas.bind("<B1-Motion>", draw)
canvas.pack()

btnClear = tk.Button(root, text = "Clear", command = clearCanvas)
btnClear.pack()

btnPredict= tk.Button(root, text = "Predict", command = saveImage)
btnPredict.pack()

lblRandForest = tk.Label(root)
lblRandForest.pack()

lblNeuralNet = tk.Label(root)
lblNeuralNet.pack()

root.mainloop() # Run it.